In [6]:
!git clone https://github.com/KutuDev/DataCentricAIComp  # colne github repo

Cloning into 'DataCentricAIComp'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 66 (delta 31), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (66/66), done.


In [7]:
def unzipper(in_folderPath, out_folderPath):  # folder unzip function
    import tarfile

    file = tarfile.open(in_folderPath)
    file.extractall(out_folderPath)
    file.close()

In [15]:
unzipper(in_folderPath='/content/DataCentricAIComp/data_v3.tar.gz', out_folderPath='./images')  #unzip files from github into the notebook

In [52]:
import os
from PIL import Image
import matplotlib.pyplot as plt
from pathlib import Path

In [65]:
 pat=("/content/images/Users/Samuel/Desktop/data_v3/train")  #the path where the data set is stored on my google colab notebook (for the train set)
 for filename in os.listdir(pat): #getting the folders in the file path
   folder=os.path.join(pat,filename)  #joining the folder names to the file path so we can go into the folders
   os.mkdir("/content/drive/MyDrive/Ai_projects/data centric ai /data_v4/train/"+ filename)  #making a directory to store the uninverted images in my google drive account,
   for files in os.listdir(folder): #getting the files in the folders
    file=os.path.join(folder,files)  #adding the file names to the file path so our uninvert function can access them directly uninvert directly
    im=Image.open(file) # our images are accessed directly
    im_invert = ImageOps.invert(im)  # images are uninverted
    im_invert.save("/content/drive/MyDrive/Ai_projects/data centric ai /data_v4/train/"+ filename + '/' + files) #uninverted images are stored in the folder i created in my google drive account

In [78]:
pat_val=("/content/images/Users/Samuel/Desktop/data_v3/val")  #repeated the procedures above for the val set changing only the file paths.
for filename in os.listdir(pat_val):
   folder=os.path.join(pat,filename)
   os.mkdir("/content/drive/MyDrive/Ai_projects/data centric ai /data_v4/val/" + filename)
   for files in os.listdir(folder):
    file=os.path.join(folder,files)
    im=Image.open(file)
    im_invert = ImageOps.invert(im)
    im_invert.save("/content/drive/MyDrive/Ai_projects/data centric ai /data_v4/val/"+ filename + '/' + files)

In [55]:
'''for filename in os.listdir(path):
  k=os.path.join(path, filename)
  im=Image.open(k)
  im_invert = ImageOps.invert(im)
  im_invert.save("/content/images/Users/Samuel/Desktop/data_v3/train_1/i/"+ filename)'''